In [69]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping

print('Libraries imported.')

Libraries imported.


In [70]:
# To run this, you can install BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# Or download the file
# http://beautiful-soup-4
# and unzip it in the same directory as this file
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')


BeautifulSoup  & csv imported.


In [72]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [73]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')

soup ready


In [76]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [79]:
table_rows = table.find_all('tr')

#table_rows

In [80]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

#print(df.head(5))
#print('***')
#print(df.tail(5))

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211 entries, 3 to 287
Data columns (total 3 columns):
PostalCode       211 non-null object
Borough          211 non-null object
Neighbourhood    211 non-null object
dtypes: object(3)
memory usage: 6.6+ KB


In [102]:
df.shape

(211, 3)

Data Cleaned

import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

df.head(15)

## Ignore cells with 'Not assigned' Borough

In [86]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head(15)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [87]:
df1 = df.reset_index()
#df1.head()

In [88]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
index            211 non-null int64
PostalCode       211 non-null object
Borough          211 non-null object
Neighbourhood    211 non-null object
dtypes: int64(1), object(3)
memory usage: 6.7+ KB


In [89]:
df1.shape

(211, 4)

# Regrouping

In [93]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

#df2.head()
       

In [94]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [96]:
df3 = df2.reset_index()
#df3.head()

# Removing Duplicates

In [97]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [99]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [100]:
df3.info()
df3.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


(103, 3)

# Using the Geocoder Package

In [164]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="London"
country ="Uk"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


latitude is :- 51.5073219 
longtitude is:- -0.1276474


In [105]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 112665056, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [107]:
import pandas as pd
#df3.head()

In [108]:
import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

from geopy.geocoders import Nominatim
geolocator = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [111]:
df_geopy1 = df3
#df_geopy1

In [112]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

df_geopy1['address'] = df3[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Rouge,Malvern","M1B, Scarborough, Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E, Scarborough, Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [113]:
df_geopy1 = df3

In [114]:
df_geopy1.shape

(103, 4)

In [115]:
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 3.3+ KB


In [116]:
df_geopy1.drop(df_geopy1[df_geopy1['Borough']=="Notassigned"].index,axis=0, inplace=True)
#df_geopy1
# code holds true up until i=102
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [118]:
#df_geopy1.head()

In [ ]:
df_geopy1.shape

In [ ]:
df_geopy1.to_csv('geopy1.csv')
# no data for location after row 75

In [127]:
#Lets test for a SINGLE LOCATION
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("M1G, Scarborough, Woburn")


#print(location.address)

#print((location.latitude, location.longitude))

print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


{'place_id': 4972888, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 558715617, 'boundingbox': ['43.7597743', '43.7598743', '-79.2253408', '-79.2252408'], 'lat': '43.7598243', 'lon': '-79.2252908', 'display_name': 'Woburn, Scarborough—Guildwood, Scarborough, Toronto, Golden Horseshoe, Ontario, M1H 2A2, Canada', 'class': 'place', 'type': 'neighbourhood', 'importance': 0.582352868045614}


In [128]:
df3.to_csv('geopy.csv')

In [129]:
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     #for row in reader:
         #print(row['PostalCode'],row['Borough'], row['Neighbourhood'] )

In [131]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("M1B Scarborough Rouge,Malvern")

#print(location.address)

#print((location.latitude, location.longitude))

print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


{'place_id': 4979177, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 558715616, 'boundingbox': ['43.8091455', '43.8092455', '-79.2217508', '-79.2216508'], 'lat': '43.8091955', 'lon': '-79.2217008', 'display_name': 'Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada', 'class': 'place', 'type': 'neighbourhood', 'importance': 0.7914299961151839}


In [134]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, Highland Creek")

#print(location.address)

print((location.latitude, location.longitude))

print(location.raw)

#M1C Scarborough Highland Creek,Rouge Hill,Port Union = no address

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


(43.7901172, -79.1733344)
{'place_id': 332533, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 139354747, 'boundingbox': ['43.7900672', '43.7901672', '-79.1733844', '-79.1732844'], 'lat': '43.7901172', 'lon': '-79.1733344', 'display_name': 'Highland Creek, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1C 3T7, Canada', 'class': 'place', 'type': 'neighbourhood', 'importance': 0.6726559169652601}


In [135]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, Highland Creek")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

#M1C Scarborough Highland Creek,Rouge Hill,Port Union = no address

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


# Create csv file

In [136]:
import numpy as np
import csv


PostalCode = None
Borough = None
Neighbourhood = None
latData = None
longData = None

LAT_Woburn = 43.7598243
LONG_Woburn = -79.2252908
LAT_Malvern = 43.8091955
LONG_Malvern = -79.2217008
LAT_Highland_Creek = 43.7901172
LONG_Highland_Creek = -79.1733344
LAT_Morningside = 43.7826012
LONG_Morningside = -79.2049579

 
PostalCode = np.array(['M1H','M1B','M1C','M1G '])
Borough = np.array(['Scarborough','Scarborough','Scarborough','Scarborough'])
Neighbourhood = np.array(['Woburn','Malvern','Highland_Creek','Morningside'])
latData = np.array([43.7598243,43.8091955, 43.7901172 , 43.7826012])
longData = np.array([-79.2252908,-79.2217008,-79.1733344, -79.2049579 ])

with open('data.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow('ABXYZ')
    for a,b,x,y,z in np.nditer([ PostalCode.T, Borough.T, Neighbourhood.T, latData.T, longData.T], order='C'):
        writer.writerow([a,b,x,y,z])    

In [137]:
import csv

with open('data.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         print(row['A'], row['B'],row['X'], row['Y'], row['Z'])

M1H Scarborough Woburn 43.7598243 -79.2252908
M1B Scarborough Malvern 43.8091955 -79.2217008
M1C Scarborough Highland_Creek 43.7901172 -79.1733344
M1G  Scarborough Morningside 43.7826012 -79.2049579


In [138]:
pd.read_csv('data.csv')

,A,B,X,Y,Z
0,M1H,Scarborough,Woburn,43.759824,-79.225291
1,M1B,Scarborough,Malvern,43.809196,-79.221701
2,M1C,Scarborough,Highland_Creek,43.790117,-79.173334
3,M1G,Scarborough,Morningside,43.782601,-79.204958


# Retrieved cordinates

In [139]:
import pandas, os
os.listdir()

['data.csv', 'geopy.csv']

In [140]:
df_geopy=df3
#df_geopy.head()

In [141]:
import geopy
#dir(geopy)

In [142]:
type(df_geopy)

pandas.core.frame.DataFrame

In [143]:
df_geopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [144]:
from geopy.geocoders import Nominatim
print('Nominatim imported')

Nominatim imported


In [145]:
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae"


In [146]:
nom = Nominatim()
n=nom.geocode('M1B, Scarborough, Rouge,Malvern')
n

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


Location(Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada, (43.8091955, -79.2217008, 0.0))

In [147]:
n.latitude

43.8091955

In [148]:
type(n)

geopy.location.Location

In [150]:
n2=nom.geocode('M1E Scarborough Guildwood,Morningside,West Hill')
print(n2)

None


In [152]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern","(Malvern, Scarborough—Rouge Park, Scarborough,..."
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port...",None
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill",None
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T..."
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None


In [153]:
df_geopy.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
Coordinates      5 non-null object
dtypes: object(5)
memory usage: 4.8+ KB


In [165]:
df_geopy.Coordinates[0]
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head(15)

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern","(Malvern, Scarborough—Rouge Park, Scarborough,...",43.809196,-79.221701
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port...",None,NaN,NaN
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill",None,NaN,NaN
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T...",43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None,NaN,NaN
5,M1J,Scarborough,Scarborough Village,"M1J,Scarborough,Scarborough Village",None,NaN,NaN
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park","M1K,Scarborough,East Birchmount Park,Ionview,K...",None,NaN,NaN
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge","M1L,Scarborough,Clairlea,Golden Mile,Oakridge",None,NaN,NaN
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West","M1M,Scarborough,Cliffcrest,Cliffside,Scarborou...",None,NaN,NaN
9,M1N,Scarborough,"Birch Cliff,Cliffside West","M1N,Scarborough,Birch Cliff,Cliffside West",None,NaN,NaN


In [155]:
df_geopy.to_csv('geo_loc_py.csv')

In [156]:
print('The latitude of', df_geopy.address[0],  'is', df_geopy.latitude[0], 'and its longitude is',df_geopy.longitude[0])

The latitude of M1B,Scarborough,Rouge,Malvern is 43.8091955 and its longitude is -79.2217008


In [157]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data2 = pd.read_csv("geopy.csv") 
# Preview the first 5 lines of the loaded data 
data2.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,address
0,0,M1B,Scarborough,"Rouge,Malvern","M1B, Scarborough, Rouge,Malvern"
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E, Scarborough, Guildwood,Morningside,West Hill"
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [177]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-2fe2d57c-baa7-4a0e-90d6-dce2bc2584b8',
    'IBM_API_KEY_ID': 'VJJvb2e79Fn4VM3k0VYsgqfpgI3qgZJ9yN3bp82bSlmN',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'capstone-donotdelete-pr-srsrfzpbewu5mm',
    'FILE': 'Geospatial_Coordinates.csv'
}
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_81df77ce01624ebda557056ce097d71d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='VJJvb2e79Fn4VM3k0VYsgqfpgI3qgZJ9yN3bp82bSlmN',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_81df77ce01624ebda557056ce097d71d.get_object(Bucket='capstone-donotdelete-pr-srsrfzpbewu5mm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

In [178]:
data3 = pd.read_csv("Geospatial_Coordinates.csv") 
# Preview the first 5 lines of the loaded data 
data3.head()

FileNotFoundError: [Errno 2] File b'Geospatial_Coordinates.csv' does not exist: b'Geospatial_Coordinates.csv'